## Authenticate to Google Earth Engine

In [1]:
import ee
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='ee-raman')

## Take user input on ROI

In [2]:
top_left = [10.62060477, 77.87217496]  # Replace lon1 and lat1 with actual values
bottom_right = [10.57740888, 77.91612074]  # Replace lon2 and lat2 with actual values

# Create a rectangle geometry using the defined corners
rectangle = ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])
print("Area of the Rectangle is ", rectangle.area().getInfo()/1e6)
# Create a feature collection with the rectangle as a boundary
roi_boundary = ee.FeatureCollection([ee.Feature(rectangle)])
filename_prefix = 'Area_testing_1'


Area of the Rectangle is  23.070494296156163


In [3]:
top_left = [20.56149133, 73.56330458]  # Replace lon1 and lat1 with actual values
bottom_right = [20.47917109, 73.60725362]  # Replace lon2 and lat2 with actual values

# Create a rectangle geometry using the defined corners
rectangle = ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])
print("Area of the Rectangle is ", rectangle.area().getInfo()/1e6)
# Create a feature collection with the rectangle as a boundary
roi_boundary = ee.FeatureCollection([ee.Feature(rectangle)])
filename_prefix = 'Area_surgana'


Area of the Rectangle is  41.89448898377194


In [4]:
top_left = [19.26903337, 80.86453214]  # Replace lon1 and lat1 with actual values
bottom_right = [19.22754399, 80.90847793]  # Replace lon2 and lat2 with actual values

# Create a rectangle geometry using the defined corners
rectangle = ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])
print("Area of the Rectangle is ", rectangle.area().getInfo()/1e6)
# Create a feature collection with the rectangle as a boundary
roi_boundary = ee.FeatureCollection([ee.Feature(rectangle)])
filename_prefix = 'Area_forested'


Area of the Rectangle is  21.28347936892187


In [5]:
top_left = [21.11059511, 73.20190113]  # Replace lon1 and lat1 with actual values
bottom_right = [21.07414383, 73.23977287]  # Replace lon2 and lat2 with actual values

# Create a rectangle geometry using the defined corners
rectangle = ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])
print("Area of the Rectangle is ", rectangle.area().getInfo()/1e6)
# Create a feature collection with the rectangle as a boundary
roi_boundary = ee.FeatureCollection([ee.Feature(rectangle)])
filename_prefix = 'Area_plantation'


Area of the Rectangle is  15.925068062492686


In [6]:
def get_feature_collection(asset_id):
    """Check if an asset exists, and load it as a FeatureCollection if it does.
    Otherwise, return an empty FeatureCollection.
    
    Args:
        asset_id (str): The Earth Engine asset ID.
        
    Returns:
        ee.FeatureCollection: The loaded FeatureCollection or an empty one.
    """
    try:
        # Get asset information to check existence
        ee.data.getAsset(asset_id)
        print(f"Asset '{asset_id}' exists. Loading FeatureCollection.")
        return ee.FeatureCollection(asset_id)
    except Exception as e:
        print(f"Asset '{asset_id}' does not exist. Returning empty FeatureCollection.")
        return ee.FeatureCollection([])

In [7]:
suffix = filename_prefix.split("_")[-1]
farm = get_feature_collection("projects/ee-raman/assets/farm_" + suffix).map(lambda x: x.set("class", 0))
scrubland = get_feature_collection("projects/ee-raman/assets/scrubland_" + suffix).map(lambda x: x.set("class", 1))
plantation = get_feature_collection("projects/ee-raman/assets/plantation_" + suffix).map(lambda x: x.set("class", 2))
rest = get_feature_collection("projects/ee-raman/assets/rest_" + suffix)
ts_data = ee.Image("projects/ee-raman/assets/ts_data_" + suffix)

training_features = farm.merge(scrubland).merge(plantation)

training_data = ts_data.sampleRegions(
    collection=training_features,
    properties=['class'],
    scale=10  # Adjust scale based on resolution
)
classifier = ee.Classifier.smileRandomForest(50).train(
    features=training_data,
    classProperty='class',
    inputProperties=ts_data.bandNames()
)

classified = ts_data.classify(classifier)
def assign_mode_label(feature):
    class_values = classified.reduceRegion(
        reducer=ee.Reducer.mode(),
        geometry=feature.geometry(),
        scale=30,  # Adjust scale as per resolution
        bestEffort=True
    )
    return feature.set('predicted_class', class_values.get('classification'))

# Apply function to test features
rest_with_labels = rest.map(assign_mode_label)

Asset 'projects/ee-raman/assets/farm_plantation' exists. Loading FeatureCollection.
Asset 'projects/ee-raman/assets/scrubland_plantation' does not exist. Returning empty FeatureCollection.
Asset 'projects/ee-raman/assets/plantation_plantation' exists. Loading FeatureCollection.
Asset 'projects/ee-raman/assets/rest_plantation' exists. Loading FeatureCollection.


In [8]:
task = ee.batch.Export.table.toAsset(
    collection=rest_with_labels,
    description='My_FeatureCollection',
    assetId="projects/ee-raman/assets/rest_all_" + suffix
)

# Start the task
task.start()